Training with 1 epoch - 1024 x 512

In [1]:
# -*- coding: utf-8 -*-
import os.path
import h5py
import json
import pickle

import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
K.set_image_data_format('channels_first')

# from core.createDataset import data_processor
from core.imageGenerator import train_generator, valid_generator
from core.networks import UNET
from core.losses import charbonnier, soft_dice


def get_hdf5(filename):
    # Reading dataset
    hdf5_file = './data/' + filename

    # if not os.path.isfile(hdf5_file):
    #     data_processor()
    return h5py.File(hdf5_file, 'r')

def save_history(obj, name):
    try:
        filename = open(name + ".pickle","wb")
        pickle.dump(obj, filename)
        filename.close()
        return(True)
    except:
        return(False)

def main():

    configs = json.load(open('config.json', 'r'))
    if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

    hf = get_hdf5(configs['data']['filename'])

    x_train = hf.get('x_train')
    y_train = hf.get('y_train')
    x_valid = hf.get('x_valid')
    y_valid = hf.get('y_valid')

    nb_epochs = configs['training']['epochs']
    batch_size= configs['training']['batch_size']

    # Currenty U-net is the only implemented network. By default the size of images is 384x128.
    model = UNET((6,512,1024), configs['layer']['activation'])

    # We use ADAM optimizer and custom loss functions as 'charbonnier' and 'soft_dice'
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    loss_f    = configs['model']['loss']
    if loss_f == "soft_dice":
        loss = soft_dice
    if loss_f == "charbonnier":
        loss = charbonnier

    def PSNR(y_true, y_pred):
        max_pixel = 1.0
        return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303
    # Compile the model. We use accuracy also but it is not so good due to the high number of classes.
    model.compile(loss=loss, optimizer=optimizer, metrics=[PSNR])

    # We only save the best model and we reduce learning rate when the val_loss is not getting
    # better under 10 epoch. It is for SGD.
    callbacks = [
            ModelCheckpoint(filepath="./" + configs['model']['save_dir'] + "/" + configs['model']['file_name'] + ".hdf5", monitor='val_loss', save_best_only=True, verbose=1),
            ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
    ]

    # Train the model
    hist = model.fit_generator(
        generator=train_generator(x_train,y_train, batch_size),
        steps_per_epoch = x_train.shape[0]/batch_size,
        validation_data=valid_generator(x_valid, y_valid, batch_size),
        validation_steps= x_valid.shape[0]/batch_size,
        epochs = nb_epochs,
        callbacks=callbacks
    )

    ## Save the history
    save_history(hist, "./" + configs['model']['save_dir'] + "/" +configs['model']['file_name'])

if __name__ == '__main__':
    main()

953/953 [==============================] - 7811s 8s/step - loss: 0.0316 - PSNR: 19.9726 - val_loss: 0.0210 - val_PSNR: 22.2682

Epoch 00001: val_loss improved from inf to 0.02099, saving model to ./model_weights\model_weights.hdf5


Training with 1 epoch 512x265

In [1]:
# -*- coding: utf-8 -*-
import os.path
import h5py
import json
import pickle

import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
K.set_image_data_format('channels_first')

# from core.createDataset import data_processor
from core.imageGenerator import train_generator, valid_generator
from core.networks import UNET
from core.losses import charbonnier, soft_dice


def get_hdf5(filename):
    # Reading dataset
    hdf5_file = './data/' + filename

    # if not os.path.isfile(hdf5_file):
    #     data_processor()
    return h5py.File(hdf5_file, 'r')

def save_history(obj, name):
    try:
        filename = open(name + ".pickle","wb")
        pickle.dump(obj, filename)
        filename.close()
        return(True)
    except:
        return(False)

def main():

    configs = json.load(open('config.json', 'r'))
    if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

    hf = get_hdf5(configs['data']['filename'])

    x_train = hf.get('x_train')
    y_train = hf.get('y_train')
    x_valid = hf.get('x_valid')
    y_valid = hf.get('y_valid')

    nb_epochs = configs['training']['epochs']
    batch_size= configs['training']['batch_size']

    # Currenty U-net is the only implemented network. By default the size of images is 384x128.
    model = UNET((6,256,512), configs['layer']['activation'])

    # We use ADAM optimizer and custom loss functions as 'charbonnier' and 'soft_dice'
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    loss_f    = configs['model']['loss']
    if loss_f == "soft_dice":
        loss = soft_dice
    if loss_f == "charbonnier":
        loss = charbonnier

    def PSNR(y_true, y_pred):
        max_pixel = 1.0
        return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303
    # Compile the model. We use accuracy also but it is not so good due to the high number of classes.
    model.compile(loss=loss, optimizer=optimizer, metrics=[PSNR])

    # We only save the best model and we reduce learning rate when the val_loss is not getting
    # better under 10 epoch. It is for SGD.
    callbacks = [
            ModelCheckpoint(filepath="./" + configs['model']['save_dir'] + "/" + configs['model']['file_name'] + ".hdf5", monitor='val_loss', save_best_only=True, verbose=1),
            ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
    ]

    # Train the model
    hist = model.fit_generator(
        generator=train_generator(x_train,y_train, batch_size),
        steps_per_epoch = x_train.shape[0]/batch_size,
        validation_data=valid_generator(x_valid, y_valid, batch_size),
        validation_steps= x_valid.shape[0]/batch_size,
        epochs = nb_epochs,
        callbacks=callbacks
    )

    ## Save the history
    save_history(hist, "./" + configs['model']['save_dir'] + "/" +configs['model']['file_name'])

if __name__ == '__main__':
    main()

953/953 [==============================] - 2083s 2s/step - loss: 0.0316 - PSNR: 19.9175 - val_loss: 0.0214 - val_PSNR: 22.2771

Epoch 00001: val_loss improved from inf to 0.02142, saving model to ./model_weights\model_weights.hdf5


Training with 10 epoch 512x256

In [2]:
# -*- coding: utf-8 -*-
import os.path
import h5py
import json
import pickle

import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
K.set_image_data_format('channels_first')

# from core.createDataset import data_processor
from core.imageGenerator import train_generator, valid_generator
from core.networks import UNET
from core.losses import charbonnier, soft_dice


def get_hdf5(filename):
    # Reading dataset
    hdf5_file = './data/' + filename

    # if not os.path.isfile(hdf5_file):
    #     data_processor()
    return h5py.File(hdf5_file, 'r')

def save_history(obj, name):
    try:
        filename = open(name + ".pickle","wb")
        pickle.dump(obj, filename)
        filename.close()
        return(True)
    except:
        return(False)

def main():

    configs = json.load(open('config.json', 'r'))
    if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

    hf = get_hdf5(configs['data']['filename'])

    x_train = hf.get('x_train')
    y_train = hf.get('y_train')
    x_valid = hf.get('x_valid')
    y_valid = hf.get('y_valid')

    nb_epochs = configs['training']['epochs']
    batch_size= configs['training']['batch_size']

    # Currenty U-net is the only implemented network. By default the size of images is 384x128.
    model = UNET((6,256,512), configs['layer']['activation'])

    # We use ADAM optimizer and custom loss functions as 'charbonnier' and 'soft_dice'
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    loss_f    = configs['model']['loss']
    if loss_f == "soft_dice":
        loss = soft_dice
    if loss_f == "charbonnier":
        loss = charbonnier

    def PSNR(y_true, y_pred):
        max_pixel = 1.0
        return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303
    # Compile the model. We use accuracy also but it is not so good due to the high number of classes.
    model.compile(loss=loss, optimizer=optimizer, metrics=[PSNR])

    # We only save the best model and we reduce learning rate when the val_loss is not getting
    # better under 10 epoch. It is for SGD.
    callbacks = [
            ModelCheckpoint(filepath="./" + configs['model']['save_dir'] + "/" + configs['model']['file_name'] + ".hdf5", monitor='val_loss', save_best_only=True, verbose=1),
            ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
    ]

    # Train the model
    hist = model.fit_generator(
        generator=train_generator(x_train,y_train, batch_size),
        steps_per_epoch = x_train.shape[0]/batch_size,
        validation_data=valid_generator(x_valid, y_valid, batch_size),
        validation_steps= x_valid.shape[0]/batch_size,
        epochs = nb_epochs,
        callbacks=callbacks
    )

    ## Save the history
    save_history(hist, "./" + configs['model']['save_dir'] + "/" +configs['model']['file_name'])

if __name__ == '__main__':
    main()

Epoch 1/5
953/953 [==============================] - 2182s 2s/step - loss: 0.0305 - PSNR: 20.0406 - val_loss: 0.0208 - val_PSNR: 22.1609

Epoch 00001: val_loss improved from inf to 0.02080, saving model to ./model_weights\model_weights.hdf5
Epoch 2/5
953/953 [==============================] - 2172s 2s/step - loss: 0.0203 - PSNR: 22.0304 - val_loss: 0.0204 - val_PSNR: 22.1980

Epoch 00002: val_loss improved from 0.02080 to 0.02036, saving model to ./model_weights\model_weights.hdf5
Epoch 3/5
953/953 [==============================] - 2157s 2s/step - loss: 0.0189 - PSNR: 22.3287 - val_loss: 0.0687 - val_PSNR: 18.0599

Epoch 00003: val_loss did not improve from 0.02036
Epoch 4/5
953/953 [==============================] - 2139s 2s/step - loss: 0.0184 - PSNR: 22.5008 - val_loss: 0.0214 - val_PSNR: 21.6348

Epoch 00004: val_loss did not improve from 0.02036
Epoch 5/5
953/953 [==============================] - 2142s 2s/step - loss: 0.0174 - PSNR: 22.7657 - val_loss: 0.0187 - val_PSNR: 22.6609

Simple U-Net arch - 512x256 - 5 epochs

In [1]:
# -*- coding: utf-8 -*-
import os.path
import h5py
import json
import pickle

import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
K.set_image_data_format('channels_first')

# from core.createDataset import data_processor
from core.imageGenerator import train_generator, valid_generator
from core.networks import UNET
from core.losses import charbonnier, soft_dice


def get_hdf5(filename):
    # Reading dataset
    hdf5_file = './data/' + filename

    # if not os.path.isfile(hdf5_file):
    #     data_processor()
    return h5py.File(hdf5_file, 'r')

def save_history(obj, name):
    try:
        filename = open(name + ".pickle","wb")
        pickle.dump(obj, filename)
        filename.close()
        return(True)
    except:
        return(False)

def main():

    configs = json.load(open('config.json', 'r'))
    if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

    hf = get_hdf5(configs['data']['filename'])

    x_train = hf.get('x_train')
    y_train = hf.get('y_train')
    x_valid = hf.get('x_valid')
    y_valid = hf.get('y_valid')

    nb_epochs = configs['training']['epochs']
    batch_size= configs['training']['batch_size']

    # Currenty U-net is the only implemented network. By default the size of images is 384x128.
    model = UNET((6,256,512), configs['layer']['activation'])

    # We use ADAM optimizer and custom loss functions as 'charbonnier' and 'soft_dice'
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    loss_f    = configs['model']['loss']
    if loss_f == "soft_dice":
        loss = soft_dice
    if loss_f == "charbonnier":
        loss = charbonnier

    def PSNR(y_true, y_pred):
        max_pixel = 1.0
        return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303
    # Compile the model. We use accuracy also but it is not so good due to the high number of classes.
    model.compile(loss=loss, optimizer=optimizer, metrics=[PSNR])

    # We only save the best model and we reduce learning rate when the val_loss is not getting
    # better under 10 epoch. It is for SGD.
    callbacks = [
            ModelCheckpoint(filepath="./" + configs['model']['save_dir'] + "/" + configs['model']['file_name'] + ".hdf5", monitor='val_loss', save_best_only=True, verbose=1),
            ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
    ]

    # Train the model
    hist = model.fit_generator(
        generator=train_generator(x_train,y_train, batch_size),
        steps_per_epoch = x_train.shape[0]/batch_size,
        validation_data=valid_generator(x_valid, y_valid, batch_size),
        validation_steps= x_valid.shape[0]/batch_size,
        epochs = nb_epochs,
        callbacks=callbacks
    )

    ## Save the history
    save_history(hist, "./" + configs['model']['save_dir'] + "/" +configs['model']['file_name'])

if __name__ == '__main__':
    main()

Epoch 1/5
953/953 [==============================] - 1371s 1s/step - loss: 0.0248 - PSNR: 21.4604 - val_loss: 0.0212 - val_PSNR: 21.4266

Epoch 00001: val_loss improved from inf to 0.02116, saving model to ./model_weights\model_weights.hdf5
Epoch 2/5
953/953 [==============================] - 1351s 1s/step - loss: 0.0191 - PSNR: 22.4564 - val_loss: 0.0194 - val_PSNR: 22.8859

Epoch 00002: val_loss improved from 0.02116 to 0.01942, saving model to ./model_weights\model_weights.hdf5
Epoch 3/5
953/953 [==============================] - 1342s 1s/step - loss: 0.0187 - PSNR: 22.5572 - val_loss: 0.0213 - val_PSNR: 21.4858

Epoch 00003: val_loss did not improve from 0.01942
Epoch 4/5
953/953 [==============================] - 1339s 1s/step - loss: 0.0185 - PSNR: 22.6376 - val_loss: 0.0195 - val_PSNR: 22.6052

Epoch 00004: val_loss did not improve from 0.01942
Epoch 5/5
953/953 [==============================] - 1333s 1s/step - loss: 0.0183 - PSNR: 22.6886 - val_loss: 0.0199 - val_PSNR: 22.3675

In [1]:
# -*- coding: utf-8 -*-
import os.path
import h5py
import json
import pickle

import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
K.set_image_data_format('channels_first')

# from core.createDataset import data_processor
from core.imageGenerator import train_generator, valid_generator
from core.networks import UNET
from core.losses import charbonnier, soft_dice


def get_hdf5(filename):
    # Reading dataset
    hdf5_file = './data/' + filename

    # if not os.path.isfile(hdf5_file):
    #     data_processor()
    return h5py.File(hdf5_file, 'r')

def save_history(obj, name):
    try:
        filename = open(name + ".pickle","wb")
        pickle.dump(obj, filename)
        filename.close()
        return(True)
    except:
        return(False)

def main():

    configs = json.load(open('config.json', 'r'))
    if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

    hf = get_hdf5(configs['data']['filename'])

    x_train = hf.get('x_train')
    y_train = hf.get('y_train')
    x_valid = hf.get('x_valid')
    y_valid = hf.get('y_valid')

    nb_epochs = configs['training']['epochs']
    batch_size= configs['training']['batch_size']

    # Currenty U-net is the only implemented network. By default the size of images is 384x128.
    model = UNET((6,256,256), configs['layer']['activation'])

    # We use ADAM optimizer and custom loss functions as 'charbonnier' and 'soft_dice'
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    loss_f    = configs['model']['loss']
    if loss_f == "soft_dice":
        loss = soft_dice
    if loss_f == "charbonnier":
        loss = charbonnier

    def PSNR(y_true, y_pred):
        max_pixel = 1.0
        return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303
    # Compile the model. We use accuracy also but it is not so good due to the high number of classes.
    model.compile(loss=loss, optimizer=optimizer, metrics=[PSNR])

    # We only save the best model and we reduce learning rate when the val_loss is not getting
    # better under 10 epoch. It is for SGD.
    callbacks = [
            ModelCheckpoint(filepath="./" + configs['model']['save_dir'] + "/" + configs['model']['file_name'] + ".hdf5", monitor='val_loss', save_best_only=True, verbose=1),
            ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
    ]

    # Train the model
    hist = model.fit_generator(
        generator=train_generator(x_train,y_train, batch_size),
        steps_per_epoch = x_train.shape[0]/batch_size,
        validation_data=valid_generator(x_valid, y_valid, batch_size),
        validation_steps= x_valid.shape[0]/batch_size,
        epochs = nb_epochs,
        callbacks=callbacks
    )

    ## Save the history
    save_history(hist, "./" + configs['model']['save_dir'] + "/" +configs['model']['file_name'])

if __name__ == '__main__':
    main()

256/256 [==============================] - 405s 2s/step - loss: 0.0579 - PSNR: 17.9659 - val_loss: 0.1163 - val_PSNR: 15.9660

Epoch 00001: val_loss improved from inf to 0.11628, saving model to ./model_weights\model_weights.hdf5
